In [ ]:
import os
import time
import glob
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pdfplumber

In [ ]:
# PDF 다운로드 폴더 설정
download_dir = "/Users/flareon/prometheus/투자추천/크롤링/pdf"
download_txt_dir = "/Users/flareon/prometheus/투자추천/크롤링/txt"
# PDF 파일 이름 변경과 텍스트 추출
def handle_pdf(file_path, title):
    new_file_path = os.path.join(download_dir, f"{title}.pdf")
    os.rename(file_path, new_file_path)
    print(f"Renamed PDF to: {new_file_path}")
    
    # PDF 파일에서 텍스트 추출
    with pdfplumber.open(new_file_path) as pdf:
        text = ''.join(page.extract_text() for page in pdf.pages if page.extract_text())
    
    # 추출한 텍스트를 txt 파일로 저장
    txt_file_path = os.path.join(download_txt_dir, f"{title}.txt")
    with open(txt_file_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)
    
    print(f"Extracted text saved to: {txt_file_path}")
    return text

# 다운로드가 완료될 때까지 기다리는 함수
def wait_for_downloads(download_dir, timeout=30):
    seconds = 0
    while seconds < timeout:
        time.sleep(1)
        # Check if there are any .crdownload files (partial download files)
        if not any(fname.endswith('.crdownload') for fname in os.listdir(download_dir)):
            return True
        seconds += 1
    return False

# Selenium 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "plugins.always_open_pdf_externally": True
})
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    for page_number in range(11, 243):
        url = f"https://whitepaper.io/coins?page={page_number}"
        driver.get(url)
        time.sleep(2)  # 페이지 로딩 대기
    
        text_classes = driver.find_elements(By.CSS_SELECTOR, '.text-ss.sm\\:text-sm.md\\:text-base.inline-flex.space-x-2.items-center')
        first_page_links = [element.get_attribute('href') for element in text_classes]

        for link in first_page_links:
            driver.get(link)
            time.sleep(2)
						
            try:
                second_link = driver.find_element(By.CSS_SELECTOR, '.bg-blue-400.hover\\:bg-opacity-80.px-10.py-2\\.5.md\\:py-3-05.text-white.rounded-12\\.5.delay-500.transition-transform.text-sm.md\\:text-4-25')
                driver.get(second_link.get_attribute('href'))
                time.sleep(2)

                pdf_object = driver.find_element(By.CSS_SELECTOR, 'object[data]')
                pdf_url = pdf_object.get_attribute('data')
                title = driver.title

                # PDF 다운로드
                driver.get(pdf_url)
                
                # 다운로드 완료 대기
                if wait_for_downloads(download_dir):
                    # 다운로드된 PDF 파일 찾기
                    downloaded_files = glob.glob(os.path.join(download_dir, "*.pdf"))
                    if downloaded_files:
                        latest_file = max(downloaded_files, key=os.path.getctime)
                        text = handle_pdf(latest_file, title)
                
                # 첫 번째 페이지로 복귀
                driver.back()
                driver.back()
                
						#whitepaper 자료가 없는 경우가 있음, 뒤로가기
            except Exception as e:
                print(f"Error: {e}")
                driver.back()
                
finally:
    driver.quit()